In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

In [2]:
from src.data_processing.Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from pickle import dump, load
import pandas as pd
import numpy as np

from src.bias.BiasDetector import BiasDetector
from src.bias.Distance import Distance

## Importing Data and Trained Classifier

**UX**: 
- the user uploads the dataset with the features (X)
- the user uploads 
    - the sklearn model as pkl file **or**
    - a file with the predictions already created (Y). 
- the user uploads scaler and ohe (if needed)

If a model is provided, the application checks if the the provided datasets and model match in terms of column names. 

In [3]:
input_data_path = "../data/raw_data/uci-default-of-credit-card/data/data.csv"
local_path_save = '../data/mlflow_artifacts/'

In [4]:
fitted_ohe = load(open(local_path_save + '_ohe.pkl', 'rb')) 
fitted_scaler = load(open(local_path_save + '_scaler.pkl', 'rb'))

In [5]:
pp = Preprocessing(input_data_path, "default")
X, Y = pp.read_dataframe()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=420)

X_test_ohe, _, _ = pp.preprocess_for_classification(df=X_test, 
                                                fit_ohe=True, 
                                                fitted_ohe=fitted_ohe,
                                                perform_scaling=True,
                                                fitted_scaler=fitted_scaler)

In [6]:
with open("mlruns/1/1e4a0667c7a64cbe8c7b023410e5781c/artifacts/model/model.pkl", "rb") as file:
    classifier = load(file)

In [7]:
predicted_prob = classifier.predict_proba(X_test_ohe)
predicted_values = classifier.predict(X_test_ohe)

#### Definition of conditioning variables
**UX**:
- the user selects the continuous features to be categorized and used for the creation of control groups. 

The application propose splitting cuts that make the resulting discrete distribution uniform (but also other binning procedures are possible). 

In [8]:
def age_buckets(x):
    if x < 30:
        return 1
    elif x < 40:
        return 2
    else:
        return 3

X_test['age_buckets'] = X.x5_age.apply(age_buckets)

**UX**:
- the user selects from a drop down menu the variables to be used for the groups creation
- the user can select "check all the variables" and the application will use all the available discrete features

In [9]:
conditioning_variables = ['x3_education', 'x4_marriage', 'age_buckets']

In [10]:
df_with_predictions = pd.concat(
    [X_test.reset_index(drop=True), pd.Series(predicted_values)], axis=1).rename(columns={0:"predictions"})

## Bias Detection

**UX**:
- the user selects from a drop down menu the binary variable to be used for the distance calculation
- the user selects a threshold for the distance
- the user selects a Distance (if more than one are provided)

In [11]:
bd = BiasDetector()

**UX**:

The user has can select between two options:
- Option 1: distance between frequencies
- Option 2: distance from reference distribution
    - in this case, the user has to insert values for a reference distribution

## Option 1: Distance between frequencies

### Comparison of the two groups observed freqs

In [12]:
bd.compare_binary_variable_groups(
    dataframe=df_with_predictions,
    labels_variable='predictions',
    binary_variable='x2_sex',
    threshold=0.1
)

(0.061124999999999985, True)

### Comparison of the two groups observed freqs conditioning to other features

In [13]:
results = bd.compare_binary_variable_conditioned_groups(
    dataframe=df_with_predictions,
    labels_variable='predictions',
    binary_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30)

In [31]:
conditioning_variables

['x3_education', 'x4_marriage', 'age_buckets']

In [14]:
results

{'x3_education==1': (3119, 0.06643323350994396, True),
 'x3_education==3': (1499, 0.06406963470319638, True),
 'x3_education==2': (4250, 0.062267282715836814, True),
 'x3_education==4': (40, 0.6, False),
 'x3_education==5': (75, 0.37500000000000006, False),
 'x3_education==6': (14, None, 'Not enough observations'),
 'x3_education==0': (3, None, 'Not enough observations'),
 'x3_education==1 & x4_marriage==1': (1106, 0.08972370131231394, True),
 'x3_education==1 & x4_marriage==2': (2002, 0.05913687251715427, True),
 'x3_education==1 & x4_marriage==3': (10, None, 'Not enough observations'),
 'x3_education==1 & x4_marriage==0': (1, None, 'Not enough observations'),
 'x3_education==3 & x4_marriage==1': (842, 0.05583405358686261, True),
 'x3_education==3 & x4_marriage==2': (616, 0.08824637199512575, True),
 'x3_education==3 & x4_marriage==3': (26, None, 'Not enough observations'),
 'x3_education==3 & x4_marriage==0': (15, None, 'Not enough observations'),
 'x3_education==2 & x4_marriage==1':

**UX**:
- the application shows the results in two views:
    - overall results: all the computed distances
    - violations: only the results above the threshold
- for both, the user can order the results by number of obs, distance or group name

In [15]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if not v[2]}

- '2: Gender (1 = male; 2 = female).',
- '3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).',
- '4: Marital status (1 = married; 2 = single; 3 = others).'

In [16]:
# sorting the violations by number of observations belonging to that group
dict(sorted(violations.items(), key=lambda item: item[1], reverse=True))

{'x3_education==2 & x4_marriage==1 & age_buckets==2': (884,
  0.12027630050736599,
  False),
 'x3_education==1 & x4_marriage==2 & age_buckets==2': (815,
  0.17115678378003057,
  False),
 'x3_education==2 & x4_marriage==2 & age_buckets==2': (671,
  0.17713615023474183,
  False),
 'x3_education==3 & x4_marriage==2 & age_buckets==3': (215,
  0.12787212787212793,
  False),
 'x3_education==3 & x4_marriage==2 & age_buckets==2': (189,
  0.263745236799129,
  False),
 'x3_education==1 & x4_marriage==2 & age_buckets==3': (174,
  0.10270774976657332,
  False),
 'x3_education==5': (75, 0.37500000000000006, False),
 'x3_education==5 & x4_marriage==1': (45, 0.31818181818181823, False),
 'x3_education==4': (40, 0.6, False)}

#### Focus analyses on particular cases
Here we want to understand/qualify some specific violation cases.

**UX**: 
- the user selects one of the available violations
- the application returns relevant details to help understand what's going on
- the user can download the results as csv file. 

In [17]:
# Focus on x3_education==3 & x4_marriage==2 & age_buckets==2 -> 189 obs, distance 0.26
focus_df = df_with_predictions.query("x3_education==3 & x4_marriage==2 & age_buckets==2")
bd.get_frequencies_list(focus_df, 'predictions', 'x2_sex', df_with_predictions.x2_sex.unique())

[array([0.49101796, 0.50898204]), array([0.22727273, 0.77272727])]

In [33]:
focus_df.head()

,id,x1_limit_bal,x2_sex,x3_education,x4_marriage,x5_age,x6_pay_0,x7_pay_2,x8_pay_3,x9_pay_4,...,x17_bill_amt6,x18_pay_amt1,x19_pay_amt2,x20_pay_amt3,x21_pay_amt4,x22_pay_amt5,x23_pay_amt6,y_default_payment_next_month,age_buckets,predictions
49,9001,50000,1,3,2,36,0,0,0,0,...,0,2166,1000,0,0,0,0,False,2,0
95,28613,360000,2,3,2,39,1,-2,-2,-2,...,0,0,271,0,561,0,0,False,2,0
161,17980,20000,1,3,2,39,0,0,0,0,...,16322,5000,2015,3000,10005,3414,270,False,2,0
258,22446,280000,2,3,2,38,-1,-1,-1,-1,...,12316,4041,16830,19862,12098,19,7428,False,2,0
260,5344,10000,1,3,2,31,0,0,2,0,...,8516,3000,0,295,459,1500,2000,False,2,0


In [18]:
focus_df.groupby('predictions').x2_sex.value_counts(normalize=True)

predictions  x2_sex
0            1         0.508982
             2         0.491018
1            1         0.772727
             2         0.227273
Name: x2_sex, dtype: float64

## Option 2: distance from reference distribution

In [25]:
male_0_ref = 75/100
male_1_ref = 25/100

female_0_ref = 75/100
female_1_ref = 25/100

ref_distribution = [np.array([female_0_ref, male_0_ref]), np.array([female_1_ref, male_1_ref])]

### Comparison of the two groups w.r.t. the reference distribution

In [26]:
bd.compare_binary_variable_groups(
    dataframe=df_with_predictions,
    labels_variable='predictions',
    binary_variable='x2_sex',
    threshold=0.1,
    reference_distribution=ref_distribution
)

([0.362125, 0.30100000000000005], [False, False])

### Comparison of the two groups w.r.t. the reference distribution conditioning to other features

In [27]:
results = bd.compare_binary_variable_conditioned_groups(
    dataframe=df_with_predictions,
    labels_variable='predictions',
    binary_variable='x2_sex',
    conditioning_variables=conditioning_variables,
    threshold=0.1,
    min_obs_per_group=30,
    reference_distribution=ref_distribution)

In [28]:
results

{'x3_education==1': (3119,
  [0.3408776779543884, 0.2744444444444445],
  [False, False]),
 'x3_education==3': (1499, [0.34375, 0.2796803652968036], [False, False]),
 'x3_education==2': (4250,
  [0.3839551230062179, 0.3216878402903811],
  [False, False]),
 'x3_education==4': (40, [0.35, 0.25], [False, False]),
 'x3_education==5': (75,
  [0.4583333333333333, 0.41666666666666663],
  [False, False]),
 'x3_education==6': (14, None, 'Not enough observations'),
 'x3_education==0': (3, None, 'Not enough observations'),
 'x3_education==1 & x4_marriage==1': (1106,
  [0.31798457087753135, 0.2717391304347826],
  [False, False]),
 'x3_education==1 & x4_marriage==2': (2002,
  [0.3540086673889491, 0.2948717948717948],
  [False, False]),
 'x3_education==1 & x4_marriage==3': (10, None, 'Not enough observations'),
 'x3_education==1 & x4_marriage==0': (1, None, 'Not enough observations'),
 'x3_education==3 & x4_marriage==1': (842,
  [0.3904494382022472, 0.33461538461538465],
  [False, False]),
 'x3_educa

In [29]:
# selecting only combinations for which the distance is greater than the threshold 
violations = {k: v for k, v in results.items() if (not v[2][0] or not v[2][1])}

In [30]:
violations

{'x3_education==1': (3119,
  [0.3408776779543884, 0.2744444444444445],
  [False, False]),
 'x3_education==3': (1499, [0.34375, 0.2796803652968036], [False, False]),
 'x3_education==2': (4250,
  [0.3839551230062179, 0.3216878402903811],
  [False, False]),
 'x3_education==4': (40, [0.35, 0.25], [False, False]),
 'x3_education==5': (75,
  [0.4583333333333333, 0.41666666666666663],
  [False, False]),
 'x3_education==1 & x4_marriage==1': (1106,
  [0.31798457087753135, 0.2717391304347826],
  [False, False]),
 'x3_education==1 & x4_marriage==2': (2002,
  [0.3540086673889491, 0.2948717948717948],
  [False, False]),
 'x3_education==3 & x4_marriage==1': (842,
  [0.3904494382022472, 0.33461538461538465],
  [False, False]),
 'x3_education==3 & x4_marriage==2': (616,
  [0.2735404896421846, 0.31470588235294117],
  [False, False]),
 'x3_education==2 & x4_marriage==1': (2047,
  [0.4149944258639911, 0.366600790513834],
  [False, False]),
 'x3_education==2 & x4_marriage==2': (2145,
  [0.3526954177897574